# Preguntas sobre el ejemplo de clasificación de imágenes con PyTorch y MLP

## 1. Dataset y Preprocesamiento

**¿Por qué es necesario redimensionar las imágenes a un tamaño fijo para una MLP?**  
Esto es así porque una red MLP requiere que todas las entradas tengan la misma dimensión. Cada imagen se transforma en un vector de píxeles y, si las imágenes tuvieran tamaños distintos, la cantidad de valores de entrada variaría, lo que impediría que la red procese correctamente los datos.

**¿Qué ventajas ofrece Albumentations frente a otras librerías de transformación como torchvision.transforms?**  
Albumentations ofrece una mayor flexibilidad y eficiencia para aplicar transformaciones complejas sobre imágenes. Permite realizar data augmentation más realista, combinando transformaciones geométricas y de color de forma sencilla y rápida. Además, está optimizada para visión por computadora y resulta especialmente útil cuando se busca mejorar la generalización del modelo mediante augmentations más avanzadas.

**¿Qué hace A.Normalize()? ¿Por qué es importante antes de entrenar una red?**  
Ajusta los valores de los píxeles utilizando una media y una desviación estándar, llevando los datos a una escala más adecuada para el entrenamiento. Esto es importante porque ayuda a que la red neuronal aprenda de forma más estable, evitando que diferencias de escala entre los valores de entrada dificulten el proceso de optimización.

**¿Por qué convertimos las imágenes a ToTensorV2() al final de la pipeline?**  
La conversión es necesaria porque PyTorch trabaja con tensores y no con imágenes en formato NumPy. Además, esta transformación adapta el orden de las dimensiones al formato esperado por la red, dejando los datos listos para ser utilizados durante el entrenamiento y la validación.

## 2. Arquitectura del Modelo

**¿Por qué usamos una red MLP en lugar de una CNN aquí? ¿Qué limitaciones tiene?**  
Yo creo que se utiliza una MLP principalmente por simplicidad y con fines didácticos, ya que permite entender claramente el flujo de datos y el proceso de entrenamiento. Sin embargo, una MLP tiene limitaciones importantes para imágenes, ya que no aprovecha la estructura espacial de los píxeles. Esto hace que su desempeño sea en general inferior al de una CNN en problemas de visión por computadora más complejos.

**¿Qué hace la capa Flatten() al principio de la red?**  
Convierte la imagen, que originalmente es una matriz multidimensional de píxeles, en un vector unidimensional. Esto es necesario porque las capas lineales de una MLP solo pueden operar sobre vectores y no sobre matrices o tensores con múltiples dimensiones espaciales.

**¿Qué función de activación se usó? ¿Por qué no usamos Sigmoid o Tanh?**  
Se utiliza ReLU porque es simple, eficiente y ayuda a evitar problemas como el vanishing del gradiente. Otras funciones como Sigmoid o Tanh pueden saturarse fácilmente, especialmente en redes más profundas, lo que dificulta el aprendizaje.

**¿Qué parámetro del modelo deberíamos cambiar si aumentamos el tamaño de entrada de la imagen?**  
Lo que deberíamos hacer es modificar el parámetro in_features de la primera capa Linear, ya que la cantidad de píxeles de entrada será mayor y la red debe ajustarse a esa nueva dimensión.

## 3. Entrenamiento y Optimización

**¿Qué hace optimizer.zero_grad()?**  
optimizer.zero_grad() se encarga de reiniciar los gradientes acumulados de la iteración anterior. En PyTorch, los gradientes se acumulan por defecto, por lo que es fundamental “limpiarlos” antes de calcular los nuevos gradientes para evitar actualizaciones incorrectas de los pesos.

**¿Por qué usamos CrossEntropyLoss() en este caso?**  
Porque es la función de pérdida estándar para problemas de clasificación multiclase. Esta función combina internamente el softmax con cross-entropy, lo que la hace adecuada y estable para entrenar modelos de clasificación.

**¿Cómo afecta la elección del tamaño de batch (batch_size) al entrenamiento?**  
El tamaño del batch influye tanto en la estabilidad como en la eficiencia del entrenamiento. Batches grandes producen gradientes más estables pero usan más memoria, mientras que batches más pequeños introducen mayor variabilidad en los gradientes, lo que a veces favorece la generalización del modelo.

**¿Qué pasaría si no usamos model.eval() durante la validación?**  
Si no se utiliza model.eval() durante la validación, las capas tipo Dropout/BatchNorm seguirían funcionando en modo entrenamiento. Esto provocaría que las métricas de validación no reflejen correctamente el desempeño real del modelo y que los resultados sean inconsistentes.

## 4. Validación y Evaluación

**¿Qué significa una accuracy del 70% en validación pero 90% en entrenamiento?**  
Esto se relaciona con el overfitting. El modelo aprende muy bien los datos de entrenamiento, pero no logra generalizar de la misma forma cuando se enfrenta a datos nuevos, lo que se refleja en una menor performance de validación.

**¿Qué otras métricas podrían ser más relevantes que accuracy en un problema real?**  
Aveces, métricas como precision, recall y F1-score resultan más informativas que la accuracy, especialmente cuando las clases están desbalanceadas o cuando ciertos errores tienen mayor impacto que otros.

**¿Qué información útil nos da una matriz de confusión que no nos da la accuracy?**  
Nos permite ver en detalle qué clases están siendo correctamente clasificadas y cuáles se confunden entre sí. A diferencia de la accuracy, ofrece una visión más profunda/pormenorizada de los errores del modelo por clase.

**En el reporte de clasificación, ¿qué representan precision, recall y f1-score?**  
La precision indica qué proporción de las predicciones positivas fue correcta, el recall mide qué proporción de los casos reales fue detectada por el modelo y el F1-score combina ambas métricas en un único valor que busca un equilibrio.

## 5. TensorBoard y Logging

**¿Qué ventajas tiene usar TensorBoard durante el entrenamiento?**  
Permite monitorear el entrenamiento de forma visual, facilitando la detección de problemas como overfitting, estancamiento de la loss o comportamientos inestables del modelo.

**¿Qué diferencias hay entre loguear add_scalar, add_image y add_text?**  
add_scalar se utiliza para registrar valores numéricos como loss o accuracy, add_image permite visualizar imágenes o salidas del modelo y add_text sirve para guardar comentarios, configuraciones o descripciones de los experimentos.

**¿Por qué es útil guardar visualmente las imágenes de validación en TensorBoard?**  
Porque nos permite observar cómo está funcionando el modelo más allá de las métricas numéricas, ayudando a detectar errores sistemáticos o problemas en ciertas clases.

**¿Cómo se puede comparar el desempeño de distintos experimentos en TensorBoard?**  
Podemos hacer eso utilizando diferentes carpetas de logs y visualizándolas de forma conjunta en TensorBoard. Esto nos permite analizar fácilmente qué configuración produjo mejores resultados.

## 6. Generalización y Transferencia

**¿Qué cambios habría que hacer si quisiéramos aplicar este mismo modelo a un dataset con 100 clases?**  
Habría que modificar la última capa del modelo para que tenga 100 neuronas de salida y creo que también aumentar la capacidad del modelo para manejar la mayor complejidad del problema.

**¿Por qué una CNN suele ser más adecuada que una MLP para clasificación de imágenes?**  
Una CNN aprovecha la estructura espacial de las imágenes, detectando patrones locales como bordes y texturas. Esto la hace mucho más eficiente y efectiva que una MLP para tareas de visión por computadora.

**¿Qué problema podríamos tener si entrenamos este modelo con muy pocas imágenes por clase?**  
El principal problema sería el overfitting, ya que el modelo podría “memorizar” los ejemplos disponibles en lugar de aprender patrones generales que se mantengan en datos nuevos.

**¿Cómo podríamos adaptar este pipeline para imágenes en escala de grises?**  
Podríamos ajustar el pipeline para trabajar con un solo canal en lugar de tres. También, haciendo esto, necesitaríamos modificar la dimensión de entrada del modelo.

## 7. Regularización – Preguntas prácticas

**¿Qué efecto tuvo BatchNorm en la estabilidad y velocidad del entrenamiento?**  
El uso de Batch Normalization mejoró la estabilidad del entrenamiento, ya que permitió que el loss evolucione de forma más “suave” y consistente. Además permitió una convergencia más rápida al reducir la sensibilidad a la inicialización de los pesos.

**¿Cambió la performance de validación al combinar BatchNorm con Dropout?**  
Esto ayudó a una mejor performance de validación. Esto es así porque permitió reducir el overfitting y mejorar la capacidad de generalización del modelo.

**¿Qué combinación de regularizadores dio mejores resultados en tus pruebas?**  
La combinación de Batch Normalization y Dropout es la que ofrece el mejor equilibrio entre estabilidad del entrenamiento y generalización.

**¿Notaste cambios en la loss de entrenamiento al usar BatchNorm?**  
Sí, presentó una evolución más controlada y menos “fluctuante”.

## 8. Inicialización de Parámetros – Preguntas prácticas

**¿Qué diferencias notaste en la convergencia del modelo según la inicialización?**  
La estrategia de inicialización influye directamente en la velocidad y estabilidad de la convergencia. Inicializaciones adecuadas permiten un aprendizaje más rápido y consistente desde las primeras épocas.

**¿Alguna inicialización provocó inestabilidad (pérdida muy alta o NaNs)?**  
Cuando la inicialización no era adecuada, se veía una inestabilidad durante el entrenamiento, reflejada en pérdidas muy altas o valores indefinidos, especialmente en redes más profundas.

**¿Qué impacto tiene la inicialización sobre las métricas de validación?**  
Una buena inicialización facilita un proceso de aprendizaje más efectivo, lo que suele traducirse en mejores métricas de validación. Por el contrario, una mala inicialización puede limitar el desempeño final del modelo.

**¿Por qué bias se suele inicializar en cero?**  
Porque así no rompe la simetría entre neuronas y permite que el aprendizaje se centre inicialmente en los pesos, sin introducir sesgos innecesarios en la red.
